In [5]:
import pandas as pd
import numpy as np
import tables

In [14]:
train = pd.read_hdf('../input/train.h5')#[:100]
test = pd.read_hdf('../input/test.h5')#[:100]

In [17]:
print(len(train))
print(len(test))
print(len(train)/len(test))


1804874
97320
18.545766543362106


In [8]:
text = train["comment_text"]

In [10]:
text

0     This is so cool. It's like, 'would you want yo...
1     Thank you!! This would make my life a lot less...
2     This is such an urgent design problem; kudos t...
3     Is this something I'll be able to install on m...
4                  haha you guys are a bunch of losers.
5                                  ur a sh*tty comment.
6                           hahahahahahahahhha suck it.
7                                   FFFFUUUUUUUUUUUUUUU
8     The ranchers seem motivated by mostly by greed...
9     It was a great show. Not a combo I'd of expect...
10                              Wow, that sounds great.
11    This is a great story. Man. I wonder if the pe...
12       This seems like a step in the right direction.
13    It's ridiculous that these guys are being call...
14    This story gets more ridiculous by the hour! A...
15    I agree; I don't want to grant them the legiti...
16    Interesting. I'll be curious to see how this w...
17                      Awesome! I love Civil Co

In [ ]:
#loads, generate features, then cleans

#Generate features
#df = pd.concat([train.loc[:, 'qid' : 'question_text'], test], sort = 'False')

print("--- Generating non_eng")
train["non_eng"] = train["question_text"].map(lambda x: contains_non_english(x))
test["non_eng"] = test["question_text"].map(lambda x: contains_non_english(x))
print("--- Generating first_word")
train["first_word"] = train["question_text"].map(lambda x: get_first_word(x))
test["first_word"] = test["question_text"].map(lambda x: get_first_word(x))
print("--- Generating total_length (num chars)")
train['total_length'] = train['question_text'].apply(len)
test['total_length'] = test['question_text'].apply(len)
print("--- Generating capitals")
train['capitals'] = train['question_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
test['capitals'] = test['question_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))

print("--- Generating caps_vs_length")
train['caps_vs_length'] = train.apply(lambda row: get_cap_vs_length(row),axis=1)
test['caps_vs_length'] = test.apply(lambda row: get_cap_vs_length(row),axis=1)

#print("--- Generating num_exclamation_marks")
#train['num_exclamation_marks'] = train['question_text'].apply(lambda comment: comment.count('!'))
#test['num_exclamation_marks'] = test['question_text'].apply(lambda comment: comment.count('!'))

print("--- Generating num_question_marks")
train['num_question_marks'] = train['question_text'].apply(lambda comment: comment.count('?'))
test['num_question_marks'] = test['question_text'].apply(lambda comment: comment.count('?'))

print("--- Generating num_punctuation")
train['num_punctuation'] = train['question_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
test['num_punctuation'] = test['question_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))

#print("--- Generating num_symbols")
#train['num_symbols'] = train['question_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
#test['num_symbols'] = test['question_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))

print("--- Generating num_words")
train['num_words'] = train['question_text'].apply(lambda comment: len(re.sub(r'[^\w\s]','',comment).split(" ")))
test['num_words'] = test['question_text'].apply(lambda comment: len(re.sub(r'[^\w\s]','',comment).split(" ")))

print("--- Generating num_unique_words")
train['num_unique_words'] = train['question_text'].apply(lambda comment: len(set(w for w in comment.split())))
test['num_unique_words'] = test['question_text'].apply(lambda comment: len(set(w for w in comment.split())))

print("--- Generating words_vs_unique")
train['words_vs_unique'] = train['num_unique_words'] / train['num_words']
test['words_vs_unique'] = test['num_unique_words'] / test['num_words']

#print("--- Generating num_smilies")
#train['num_smilies'] = train['question_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
#test['num_smilies'] = test['question_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

print("--- Generating num_sentences")
train['num_sentences'] = train['question_text'].apply(lambda comment: len(re.split(r'[.!?]+', comment)))
test['num_sentences'] = test['question_text'].apply(lambda comment: len(re.split(r'[.!?]+', comment)))

print("--- Generating max_word_len")
train['max_word_len'] = train['question_text'].apply(lambda comment: calc_max_word_len(re.sub(r'[^\w\s]','',comment).split(" ")))
test['max_word_len'] = test['question_text'].apply(lambda comment: calc_max_word_len(re.sub(r'[^\w\s]','',comment).split(" ")))

print("cleaning text")
train["question_text"] = train["question_text"].apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_text(x))

print("remove single characters")
train["question_text"] = train["question_text"].apply(lambda x: remove_singles(x))
test["question_text"] = test["question_text"].apply(lambda x: remove_singles(x))

print("cleaning numbers")
train["question_text"] = train["question_text"].apply(lambda x: clean_numbers(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_numbers(x))

print("cleaning misspellings")
train["question_text"] = train["question_text"].apply(lambda x: replace_typical_misspell(x))
test["question_text"] = test["question_text"].apply(lambda x: replace_typical_misspell(x))

print("filling missing values")
#clean chinese, korean, japanese characters
print('cleaning characters')
train["question_text"] = train["question_text"].map(lambda x: remove_non_english(x))
test["question_text"] = test["question_text"].map(lambda x: remove_non_english(x))

## fill up the missing values
train["question_text"].fillna("").values
test["question_text"].fillna("").values

#for getting num good and bad words
from wordcloud import STOPWORDS
from collections import defaultdict
import operator

train1_df = train[train["target"]==1]
train0_df = train[train["target"]==0]

## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

freq_dict_bad = defaultdict(int)
for sent in train1_df["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict_bad[word] += 1
freq_dict_bad = dict(freq_dict_bad)

freq_dict_good = defaultdict(int)
for sent in train0_df["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict_good[word] += 1
freq_dict_good = dict(freq_dict_good)

bad_words = sorted(freq_dict_bad, key=freq_dict_bad.get, reverse=True)[:1000]
good_words = sorted(freq_dict_good, key=freq_dict_good.get, reverse=True)[:1000]

print("--- Generating num_bad_words")
train["num_bad_words"] = train["question_text"].map(lambda x: num_bad_words(x))
test["num_bad_words"] = test["question_text"].map(lambda x: num_bad_words(x))

print("--- Generating num_good_words")
train["num_good_words"] = train["question_text"].map(lambda x: num_good_words(x))
test["num_good_words"] = test["question_text"].map(lambda x: num_good_words(x))
